# Introduction

To-Go is a new business venture to get more fast food on-campus. The aim of this project is to design a program that simulates a fast-food environment. To do so, we will be use two small data files in .csv format, which will manipulate in order to get a data frame which will be used as the input of the fast-food simulator program.

## Section 1: Data loading and manipulation

The two files that are located in a dropbox:
1. `menu.csv`: A table that shows the code to be entered in the fast food menu item keyboard to get a certain food. It also contains the amount of products available. The file can be read from this address: https://www.dropbox.com/s/aer7yfyzrd6yog5/menu.csv?raw=1

2. `foods.csv`: A table with the name of each food (without capitalisation), the category and the price. The file can be read from this address: https://www.dropbox.com/s/ln97rid8o3bdh1h/foods.csv?raw=1

By running the following code cells, you can to read and print the two files as pandas data frames:

In [ ]:
import pandas as pd

In [ ]:
menu = pd.read_csv('https://www.dropbox.com/s/aer7yfyzrd6yog5/menu.csv?raw=1')
menu

In [ ]:
foods = pd.read_csv('https://www.dropbox.com/s/ln97rid8o3bdh1h/foods.csv?raw=1')
foods


### Question 1: Standardising the column names

The food names are not written exactly in the same on both data frames! Most notably, you can see that in the `foods` data frame the foods have the brand before the food name, whereas in the `menu` data frame the foods are capitalised and do not have the brand name before. Therefore, you need to create and **apply** a function which cleans the `food` entries of the `foods` data frame to make them look **the same way** as in the `menu` data frame!

In [ ]:
# Defining a function to standardize the foods in the food table
def change_name(x):
  #split the strings that have 's
  name = x.split("'s ", 1)
  return name[1].title()

#Applying the function to the foods column in the food table
foods['food'] = foods['food'].apply(change_name)

foods

### Question 2: Merging the Datasets

Once that both columns look the same, you need to **merge** them as a single data frame so that this can be used in the program.

*Hint:* The merged data frame can have `menu_number` as the index, and all the information from the `foods` data frame next to each product.

In [ ]:
## use this cell to merge foods into menu, your output should look like this
# merging the food and menu table using an outer join and setting menu_number as the index for the new table
merged_data = menu.merge(foods, on='food', how='outer').set_index('menu_number').sort_values('menu_number')

merged_data

### Question 3: Changing values in a data frame

Oops! I forgot to add three Chips in B4! Can you add them to the data frame please?

In [ ]:
## Use this cell to add three units of Chips to slot B4, output should look like this
# Adding chips to menu slot B4
merged_data.loc['B4'] = ['Chips', 3.0, 'side', 3.00]

merged_data

## Section 2: Fast-Food Simulation Program

Once that you have generated your merged dataset, you will use the following code cell to write a small program with the following *four* options:

1. **Admin login**: This option will simulate how the fast-food operator validates their credentials. When this option is selected, the user should be prompted to input their username and password. The username should be *admin* and the password should be any number smaller than *10* (including float and negative numbers). You should allow the user to try *four* times, if all of them are incorrect, then the program stops and the program cell needs to be run again.

  * Note: Once that an admin has logged in, this option has to be disabled (i.e. the user cannot go back and select this option).

2. **Add products**: This option can only be accessed once option 1 has been completed, otherwise the message *login first!* should be displayed and the program should go back to the main menu. In this option, the admin will be shown the list of foods and categories in the fast-food menu. Then, the admin can input a menu number  to add *one more product* for that certain menu slot. Afterwards, the program should show the new foods and go back to the main menu.

  * Note: Keep in mind that each space in the fast-food menu can hold a maximum of *eight* products (of the same food, of course). Also, you cannot add products in the empty menu slots. Therefore, your program should warn the admin in case that they want to add more products of a specific food in a menu slot, or if the admin wants to add products in an empty slot.

3. **Buy food**: This option can be accessed by "anyone", so there is no need for a validation. If this option is selected, the user will be shown the list of foods, amounts and prices. Then, the user will be requested to select one food item based on the `menu_number`. Once the food is selected, your program must display the price of the selected product. Then, the user will be prompted to pay. To simulate this payment, you will ask the user for an input and write any positive number (if the user inputs something invalid, ask to try again). Then, your program must check this number against the price of the product to be bought. If the input number is larger than the price, then you should return *your change is...* and the subtraction of the payment minus the price of the snack. If the number is equal to the price, then you must return *thanks for paying*. Else, you should output *you need to pay more* and allow the user to write another amount. After the "purchase", you should output to the user the number of products left for that particular food (therefore, you need to update the data frame!). Notice that if there were zero products left for the food selected, then you must prompt this to the user in advance before letting them buy, and ask them to select another food.

4. **Exit**

In [ ]:
# ------- Question 1 -------
## Defining a function to simulate admin login. The user has 4 attempts after which the code breaks
def admin_login():
  login_attempts = 4
  #Referencing the logged_in and loop variables
  global logged_in
  global loop
  while login_attempts > 0 and not logged_in:
    try:
      # get user input to enter password and username
      adminUsername = input("\nEnter username: ")
      adminPassword = float(input("\nEnter password: "))
      # Set admin login credentials. password = any number less than 10 (it takes float and negative values), username is set to 'admin' (upper or lowercase)
      if adminUsername.lower() == "admin" and (adminPassword < 10):
        logged_in = True
        print("\nLogin successful! \U0001f44d")
        break
      else:
        # Warining to user if they input wrong login details
        login_attempts -= 1
        print(f"\nInvalid username or password. You have {login_attempts} attempts left. \U0001F615")
    except ValueError:
        login_attempts -= 1
        print(f"\nInvalid username or password. You have {login_attempts} attempts left. \U0001F615")
    # The program stops once the user has used up all four login attempts. User will have to run cell again
    if login_attempts == 0:
      loop = False
      print("\nToo many login attempts. Login disabled. Run program cell again \U0001F644")
      break

# ------- Question 2 -------
## Defining a function to allow the admin to add product to a menu slot
def add_products():
    global merged_data
    print('\nFood Menu')
    print(merged_data)

    select_menu_no = input('\nSelect menu number to add one more product: ').upper()
    # Warning if a user inputs a menu number that is not in the DataFrame
    if select_menu_no not in merged_data.index:
      print("\nInvalid menu number \U0001F615")
      return add_products()
    # Warning message to the user if the amount in a certain food menu slot is = 8
    elif merged_data.loc[select_menu_no]['amount'] == 8:
        print('\n This menu slot is full! space in the fast-food menu can hold a maximum of eight products. Try again \U0001F622')
        return add_products()
    # Warning to user if they want to add product to an empty menu slot
    elif not merged_data.loc[select_menu_no].notna().all():
      print("\nCannot add product in an empty slot. Try again \U0001F622")
      return add_products()
    # Increase the amount of a certain food by 1
    else:
      merged_data.at[select_menu_no, 'amount'] += 1
      print(merged_data)

#------- Question 3 -------
## Defining a function to get valid payment input
def get_valid_payment():
  while True:
    payment = input("\nPlease enter payment amount: ")
    try:
      # Ensuring user only inputs positive numeric values
      payment = float(payment)
      if payment < 0:
        raise ValueError
      return payment
    except ValueError:
      print("\nInvalid input. Please enter a positive number \U0001F615")

## Defining a function to allow user buy a certain food
def buy_food():
  print('\nFood Menu')
  food_menu = merged_data.loc[merged_data['food'].notnull(), ['food', 'amount', 'price']]
  print(food_menu)

  # Get the selected food item
  select_food = input("\nPlease select a food item by menu number: ").upper()
  # Warning if the user inputs a menu number that is not in the DataFrame
  if select_food not in food_menu.index:
    print(f"\nInvalid menu number {select_food}, please try again \U0001F615")
    return buy_food()
  # Warning if the amount in a certain menu slot is 0, it tells the user that it is out of stock
  elif food_menu.loc[select_food]['amount'] == 0:
    print('\nOut of Stock! Please buy something else \U0001F61E')
    return buy_food()
  else:
    #Else it prints the food and price of the food
    food_price = food_menu.loc[select_food]['price']
    print(f"\nThe price of {food_menu.loc[select_food]['food']} is £{food_price}")
    # Get the payment amount and handle change
    payment = get_valid_payment()
    # Return the change amount in 2 decimal places
    change = round(payment - food_price, 2)
    if change > 0:
      print(f"\nYour change is £{change}.")
    # Warning if the user inputs an amount less than the food price User has to renter an amount equal or greater than food price
    else:
      while change < 0:
        print(f"\nYou need to pay more! Amount inputed is less than £{food_price} \U0001F914")
        payment = get_valid_payment()
        change = round(payment - food_price, 2)
        if change > 0:
          print(f"\nYour change is £{change}.")

    # Update the food inventory
    merged_data.at[select_food, 'amount'] -= 1

    print("\nThank you for paying \U0001f44d")

    print("\nThere are {} left for this product".format(merged_data.loc[select_food]['amount']))


loop = True
logged_in = False
while loop == True:
  print('\nWelcome to the To-Go fast-food menu. Select an option:')
  print('\n1. Log in (admin only)')
  print('\n2. Add one more product of a certain food (admin only)')
  print('\n3. Buy products')
  print('\n4. Exit')
  # then you ask the user for options
  user_input = input('\nPlease make a choice(1-4): ')


  # If user inputs 1, it prompts user to login as admin.
  if user_input == '1':
    # Once an admin has logged in, this option is disabled and tells the user they are already logged in
    if logged_in:
      print('\nYou are already logged in! \U0001f600')
      continue
    admin_login()
  # If user inputs 2, it prompts the user to add products. This option can only be accessed after the admin logs in
  elif user_input == '2':
    if logged_in:
      add_products()
    else:
      print('\nLogin Frist!')
  # If user inputs 3, it prompts the user to buy food. This option can be accessed by anyone
  elif user_input == '3':
    buy_food()
  # If user inputs 4, it stops the program
  elif user_input == '4':
    print('\nGoodbye! \U0001F61E')
    break
  # Warns the user if the input is invalid.
  else:
    print("\nInvalid input! Please try again \U0001F615")